In [ ]:
!nvidia-smi

# Setup & Install Ultralytics

In [ ]:
!pip install ultralytics roboflow --upgrade -q

import ultralytics
ultralytics.checks()

# Connect Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

project_dir = "/content/drive/MyDrive/Colab Notebooks/Computer Vision Workshop/License Plate Detection"
os.makedirs(project_dir, exist_ok=True)
os.chdir(project_dir)

In [ ]:
!pwd

# Load Dataset from Roboflow

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="your-api-key") # Roboflow API key
project = rf.workspace("roboflow-universe-projects").project("license-plate-recognition-rxg4e")
version = project.version(11)
dataset = version.download("yolov11")

In [ ]:
!ls

In [ ]:
dataset.location

In [ ]:
import os
import matplotlib.pyplot as plt
import cv2

# Base directory from Roboflow dataset or your own custom structure
# base_path = dataset.location  # or "/content/drive/..." if use Google Drive
base_path = "/content/drive/MyDrive/Colab Notebooks/Computer Vision Workshop/License Plate Detection/License-Plate-Recognition-11"

# Your folder names: train / valid / test
sets = ['train', 'valid', 'test']
image_paths = []

# Collect 4 images from each subset
for split in sets:
    img_dir = os.path.join(base_path, split, 'images')
    if os.path.exists(img_dir):
        all_images = sorted(os.listdir(img_dir))
        sample_images = all_images[:4]  # Get first 4 images
        image_paths.extend([os.path.join(img_dir, img) for img in sample_images])
    else:
        print(f"Folder not found: {img_dir}")

# Plot images in 3 rows and 4 columns
fig, axs = plt.subplots(3, 4, figsize=(16, 10))
fig.suptitle('Sample Images from Train / Valid / Test Sets', fontsize=18)

for i, img_path in enumerate(image_paths):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    row = i // 4
    col = i % 4
    axs[row, col].imshow(img)
    axs[row, col].set_title(os.path.basename(img_path), fontsize=10)
    axs[row, col].axis('off')

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()


# Fine Tuning using YOLOv11

---



## Resume Tuning

In [ ]:
from ultralytics import YOLO

# load lasted weight (last.pt) from /runs/{task}/...
model = YOLO(f"{project_dir}/runs/detect/license-plate-finetune-v1n/weights/last.pt")

# resume training..
model.train(
    resume=True
)


## Fine Tuning Model

In [ ]:
from ultralytics import YOLO

model = YOLO("yolo11n.pt")

model.train(
    data=f"{base_path}/data.yaml",
    epochs=300,
    imgsz=640,
    batch=16,
    name="license-plate-finetune-v1n",
    device=0, # GPU
    patience=15, # early stopping if not improve
    workers=2,
    val=True
)

## Validation

In [ ]:
!pwd

In [ ]:
from ultralytics import YOLO

model = YOLO(f"{project_dir}/runs/detect/license-plate-finetune-v1n/weights/best.pt")

metrics = model.val()

## Evaluate

In [ ]:
# mAP, Precision, Recall, F1
print(metrics)

In [ ]:
from IPython.display import Image, display
results_file = f"{project_dir}/runs/detect/license-plate-finetune-v1n/results.png"
display(Image(filename=results_file))

# Convert To ONNX

In [ ]:
!pwd

In [ ]:
!pip install onnxsim

In [ ]:
model_name = ""

In [ ]:
from ultralytics import YOLO
import os
import shutil

project_dir = "/content/drive/MyDrive/Colab Notebooks/Computer Vision Workshop/License Plate Detection"
model_path = os.path.join(project_dir, f"runs/detect/{model_name}/weights/best.pt")
output_path = os.path.join(project_dir, f"runs/detect/{model_name}/weights/{model_name}.onnx")


In [ ]:
model_path

In [ ]:
output_path

In [ ]:
# load best.pt model
model = YOLO(model_path)

# export
result = model.export(format="onnx", dynamic=True, simplify=True)

# move .onnx file to output path
default_output = result  # default is .onnx path
shutil.move(default_output, output_path)